In [3]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import pandas as pd
from imblearn.over_sampling import SMOTE
plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)

In [4]:
ball_df = pd.read_csv('../Data/ballbyball.csv')
country_df = pd.read_csv('../Data/country.csv')
ground_df = pd.read_csv('../Data/ground.csv')
matches_df = pd.read_csv('../Data/matches.csv')
players_df = pd.read_csv('../Data/players.csv')
season_df = pd.read_csv('../Data/season.csv')
team_df = pd.read_csv('../Data/team.csv')
town_df = pd.read_csv('../Data/town.csv')

In [5]:
ball_df['totalRuns'] = ball_df['totalRuns'] - ball_df['run']
ball_df['totalWickets'] = ball_df['totalWickets'] - ball_df['isWicket'].apply(lambda x: 1 if x else 0)

In [6]:
ground_df.drop(['Ground Slug', 'Town ID', 'Capacity'], axis=1, inplace=True)

In [7]:
matches_df.rename(columns={'ground_id': 'Ground ID'}, inplace=True)

matches_df = pd.merge(matches_df, ground_df, on='Ground ID', how='inner')

In [8]:
matches_df.drop(['season_id', 'slug', 'title', 'date', 'time', 'status', 'status_description', 'winner_team_id', 'toss_winner_team_id', 'toss_choice', 'Ground ID', 'team_1_id', 'team_2_id', 'team_1_score', 'team_1_scoreInfo', 'team_2_scoreInfo'], axis=1, inplace=True)

In [9]:
players_df.drop(['Full Name', 'Image', 'DOB', 'Batting Style (s)', 'Bowling Style (s)', 'Team ID'], axis=1, inplace=True)

In [10]:
ball_df = pd.merge(ball_df, matches_df, on='match_id', how='inner')

In [11]:
ball_df.drop(['index', 'overNumber', 'ballNumber', 'oversUnique', 'penalties', 'batsmanRuns', 'outPlayerId'], axis=1, inplace=True)

In [12]:
ball_df = ball_df.merge(players_df, how='left', left_on='batsmanPlayerId', right_on='ID', suffixes=('', '_batsman'))

ball_df.rename(columns={
    'Name': 'Batsman_Name',
    'Role': 'Batsman_Role',
    'Batting Style (l)': 'Batsman_Batting_Style',
    'Bowling Style(l)': 'Batsman_Bowling_Style',
    'Playing Role': 'Batsman_Playing_Role'
}, inplace=True)

ball_df.drop(columns=['ID'], inplace=True)

ball_df = ball_df.merge(players_df, how='left', left_on='bowlerPlayerId', right_on='ID', suffixes=('', '_bowler'))

ball_df.rename(columns={
    'Name': 'Bowler_Name',
    'Role': 'Bowler_Role',
    'Batting Style (l)': 'Bowler_Batting_Style',
    'Bowling Style(l)': 'Bowler_Bowling_Style',
    'Playing Role': 'Bowler_Playing_Role'
}, inplace=True)

ball_df.drop(columns=['ID'], inplace=True)

In [13]:
ball_df.drop(columns=['batsmanPlayerId', 'bowlerPlayerId'], inplace=True)

In [14]:
ball_df.drop(columns=['Batsman_Bowling_Style', 'Bowler_Batting_Style'], inplace=True)

In [15]:
df = ball_df.dropna(subset=['pitchLength', 'pitchLine','shotType','Batsman_Name','Batsman_Batting_Style', 'Batsman_Playing_Role','Bowler_Name','Bowler_Bowling_Style','Bowler_Playing_Role'])

In [16]:
df.drop(columns=['match_id'], inplace=True)

/tmp/ipykernel_127915/690851263.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['match_id'], inplace=True)


In [17]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
X = df.drop('isWicket', axis=1)
y = df['isWicket']

categorical_cols = X.select_dtypes(include='object').columns
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

columns_to_scale = [col for col in X.columns if col not in ['pitchLine', 'pitchLength', 'Bowler_Bowling_Style', 'shotType']]

scaler = StandardScaler()
X_resampled[columns_to_scale] = scaler.fit_transform(X_resampled[columns_to_scale])

df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
df_resampled['isWicket'] = y_resampled

In [18]:
import pickle
import joblib
with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']